### Purpose of this notebook: Develop algorithms to procedurally generate block construction targets 

* blockworld is 8x8 grid
* allowable blocks are: 1x1, 2x2, 4x4 squares and the triangles that comprise them
* target difficulty roughly varies according to shape and number of holes
* constraints on construction include: 
    * physical stability (a tipped over triangle will fall!)
    * geometry (presence & shape of "holes")
    * inventory (not infinite number of large 8x2 blocks!)
    * cost (# blocks)

In [ ]:
from __future__ import division

import numpy as np
import os, sys
from PIL import Image

from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.path import Path
import matplotlib.patches as patches
%matplotlib inline

from IPython.core.pylabtools import figsize, getfigs

import seaborn as sns

import random

from scipy.stats import norm
from IPython.display import clear_output

import copy
import importlib

### Add Paths

## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'..','..')) ## use relative paths

## add helpers to python path
import sys
if os.path.join(proj_dir, 'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'stimuli'))

## custom helper modules
import separation_axis_theorem as sat
import blockworld_helpers as utils

def cls():
    os.system('cls' if os.name=='nt' else 'clear')


### start with simple cases 

#### define inventory of block types, positioned at origin

In [ ]:
## block primitives

#### squares
s1 = np.array([(0, 0), (0, -1), (1, -1), (1, 0), (0,0)]) ## 1x1
s2 = s1*2 ## 2x2
s4 = s1*4 ## 4x4

In [ ]:
## render two blocks side by side
patch1 = utils.get_patch(s4)
patch2 = utils.get_patch(s2-2)
patch3 = utils.get_patch((utils.translate(s4, 2, 2)))

patches = [patch1,patch2,patch3]

utils.render_blockworld(patches)

In [ ]:
## distinguish between two blocks merely touching & actually coinciding 
importlib.reload(sat)
print(sat.separating_axis_theorem(s2[:-1]-2,s4[:-1])) ## 
print(sat.apply_sat(s2[:-1]-2,s4[:-1]))

In [ ]:
## sketch of blocklaying procedure:

## sample block size and place in leftmost, lowest position that you can and place in the array

## make sure that no blocks collide

## keep going until the summed area of all blocks equals or exceeds the total area of the arena

## fill these crevices

In [ ]:
importlib.reload(utils)
b1 = utils.Block(1,1)
b2 = utils.Block(2,2)
b3 = utils.Block(4,2)

In [ ]:
blocks = [b1,b2,b3]
floor_space = 8

def fill_floor(blocks, floor_space):
    '''
    Fills a level horizontal surface with blocks.
    Input: Lexicon of blocks- np arrays with 5 coordinates; length of available floor space
    Output: List of blocks that can be used to fill the floor space with no gaps
    '''
    block_widths = list(map(lambda b: b.width, blocks))

    floor_blocks = []
    floor_block_widths = []
    viable_block_widths = copy.deepcopy(block_widths)
    remaining_space = floor_space
    while remaining_space > 0:
        i = np.random.randint(0,len(viable_block_widths))
        if block_widths[i] <= remaining_space:
            floor_blocks.append([blocks[i],floor_space-remaining_space])
            floor_block_widths.append(block_widths[i])
            remaining_space -= block_widths[i]
        else:
            viable_block_widths.pop()
    #print(floor_blocks)
    #print(floor_block_widths)
    return(floor_blocks)

# visualize blocks placed on floor


def patch_for_block_here(b, x, y):
    return utils.get_patch(b.translate(b.verts,x,y))

def patches_for_floor(floor_blocks):
    patches = []
    for block_and_x in floor_blocks:
        patches.append(patch_for_block_here(block_and_x[0],block_and_x[1],0))
    return patches


In [ ]:
## Helper functions needed
# Calculate 'floors' to be placed on
# Recursively call fill_floor until whole square filled
# Create World Class

In [ ]:
# Visualize blocks placed on floor

blocks = [b1,b2,b3]
floor_space = 8
floor_blocks = fill_floor(blocks, floor_space)
utils.render_blockworld(patches_for_floor(floor_blocks))

In [ ]:
# calculate 'floors' to be placed on
# traverse bitmap of world left to right, bottom to top. 
    # When find space, start counting floor. Stop when block or at end.
    # Call fill_floor
    #



In [ ]:
def patch_for_block_here(b, x, y):
    return utils.get_patch(b.translate(b.verts,x,y))

def patches_for_floor(floor_blocks, xs):
    patches = []
    for (i, b) in enumerate(floor_blocks):
        patches.append(patch_for_block_here(b,xs[i],0))
    return patches

def drawFloor(floor_blocks, xs):
    utils.render_blockworld(patches_for_floor(floor_blocks, xs))
    
    
def patches_for_world(blocks_and_locations):
    patches = []
    for (b, x, y) in blocks_and_locations:
        patches.append(patch_for_block_here(b,x,y))
    return patches

def drawWorld(world):
    utils.render_blockworld(patches_for_world(world.blocks))

# World Class
class World:
    width = 8
    height = 8

    b1 = utils.Block(1,1)
    b2 = utils.Block(1,2)
    b3 = utils.Block(2,1)
    b4 = utils.Block(2,2)
    b5 = utils.Block(4,2)

    block_types = [b1, b2, b3, b4, b5] # Block types should be in order from left to right, thinest to thickest, shortest to tallest
    block_widths = list(map(lambda b: b.width, block_types))
    
    def __init__(self):
        
        # Actual blocks present in world
        
        self.block_map = np.zeros((World.width, World.height), dtype=int) #bitmap for placement of blocks
        
        self.blocks = []
        
        self.full = False
        
    def check_full(self):
        '''
        Checks to see whether the World contains any empty space by summing block_map
        '''
        if not self.full:
            isFull = (sum(sum(self.block_map)) == width*height)
            self.full = isFull
            return isFull
        else:
            return True
        
    def fill_floor(self, floor_space):
        '''
        Fills a 'floor', a level horizontal surface, with blocks.
        Input: Lexicon of blocks- np arrays with 5 coordinates; length of available floor space
        Output: List of blocks that can be used to fill the floor space with no gaps
        '''
        
        floor_blocks = []
        floor_block_widths = []
        viable_block_widths = copy.deepcopy(self.block_widths)
        remaining_space = floor_space
        while remaining_space > 0:
            i = np.random.randint(0,len(viable_block_widths))
            if self.block_widths[i] <= remaining_space:
                floor_blocks.append([self.block_types[i],floor_space-remaining_space])
                floor_block_widths.append(self.block_widths[i])
                remaining_space -= self.block_widths[i]
            else:
                viable_block_widths.pop()
        return(floor_blocks)

    def fill_floor_here(self, lvl, left, right):
        '''
        Fills a 'floor', a level horizontal surface, with blocks.
        Input: Lexicon of blocks- np arrays with 5 coordinates; length of available floor space
        Output: List of blocks that can be used to fill the floor space with no gaps
        '''
        
        floor_space = right-left
        floor_blocks = []
        floor_block_xs = [left]
        
        viable_block_widths = copy.deepcopy(self.block_widths)
        viable_blocks = copy.deepcopy(self.block_types)
        
        remaining_height = World.height - lvl
        remaining_space = floor_space
        
        while remaining_space > 0:
            i = np.random.randint(0,len(viable_blocks))
            block_type = self.block_types[i]
            block_width = block_type.width
            block_height = block_type.height
            if block_width <= remaining_space and block_height <= remaining_height: 
                floor_blocks.append(self.block_types[i])
                self.blocks.append((block_type,floor_block_xs[-1],lvl))
                floor_block_xs.append(floor_block_xs[-1] + block_width) # x location of bottom left of blocks
                remaining_space -= block_width
            else:
                viable_blocks.pop()
        self.update_map_with_floor_blocks(lvl, floor_blocks, floor_block_xs)
        return(floor_blocks, floor_block_xs)

    def update_map_with_floor_blocks(self, lvl, floor_blocks, floor_block_xs):
        for (i, b) in enumerate(floor_blocks):
            self.block_map[self.height-(lvl+b.height): self.height-lvl, floor_block_xs[i]:floor_block_xs[i+1]] = 1  
        #print(self.block_map)
    
    def fill_world(self):
        '''
        Semi-randomly fills world with blocks, adding a 'floor' of blocks to the next available flat surface 
        '''
        lvl = 0 #Start at bottom and work up
        while lvl <= World.height - 1:
            #find floor
            while self.block_map[World.height - lvl - 1].all() and lvl < World.height: # check if level is full or reached top
                lvl += 1
            if lvl == World.height:
                    break
            left = 0
            while self.block_map[World.height - lvl - 1][left] == 1:
                left += 1
            right = left
            while right < World.width and self.block_map[World.height - lvl - 1][right] == 0:
                right += 1
            #print('fill_world_here: ' + str((lvl, left, right)))
            self.fill_floor_here(lvl, left, right)
        
        drawWorld(self)
    

    

In [ ]:
w = World()
w.fill_world()


In [ ]:
w = World()
(floor_blocks, xs) = w.fill_floor_here(0, 0, 8)
drawFloor(floor_blocks, xs)
floor_blocks, xs

#w.update_map_with_floor_blocks(0, floor_blocks, xs)


In [ ]:
# Better way to render blocks

In [ ]:
#block_map = np.zeros((width, height), dtype=int)
block_map = np.array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1]])
block_map[1:7, 5:7]


# TO DO: Update blocks and block_map based on fill_floor

In [ ]:
# Find the start and end points of the floor to place new blocks on
# lvl is the level above which new blocks should be placed
# left is the first free location in this floor
# right is the last free location in this floor

w = World()

lvl = 0 #Start at bottom and work up
while lvl <= World.height - 1:
    #find floor
    while w.block_map[World.height - lvl - 1].all() and lvl < World.height: # check if level is full or reached top
        lvl += 1
    if lvl == World.height:
            break
    left = 0
    while w.block_map[World.height - lvl - 1][left] == 1:
        left += 1
    right = left
    while right < World.width and w.block_map[World.height - lvl - 1][right] == 0:
        right += 1
    #print('fill_world_here: ' + str((lvl, left, right)))
    w.fill_floor_here(lvl, left, right)
    
    

In [ ]:
new_blocks = w.fill_floor_here(lvl, left, right)
drawFloor(new_blocks)